# Vector search in Python (Azure AI Search)

This code demonstrates how to use Azure AI Search by using the push API to insert vectors into your search index:

+ Create an index schema
+ Load the sample data from a local folder
+ Embed the documents in-memory using Azure OpenAI's text-embedding-ada-002 model
+ Index the vector and nonvector fields on Azure AI Search
+ Run a series of vector and hybrid queries, including metadata filtering and hybrid (text + vectors) search. 

The code uses Azure OpenAI to generate embeddings for title and content fields. You'll need access to Azure OpenAI to run this demo.

The code reads the `text-sample.json` file, which contains the input data for which embeddings need to be generated.

The output is a combination of human-readable text and embeddings that can be pushed into a search index.

## Prerequisites

+ An Azure subscription, with [access to Azure OpenAI](https://aka.ms/oai/access). You must have the Azure OpenAI service name and an API key.

+ A deployment of the text-embedding-ada-002 embedding model.

+ Azure AI Search, any tier, but choose a service that has sufficient capacity for your vector index. We recommend Basic or higher. [Enable semantic ranking](https://learn.microsoft.com/azure/search/semantic-how-to-enable-disable) if you want to run the hybrid query with semantic ranking.

We used Python 3.11, [Visual Studio Code with the Python extension](https://code.visualstudio.com/docs/python/python-tutorial), and the [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) to test this example.

### Set up a Python virtual environment in Visual Studio Code

1. Open the Command Palette (Ctrl+Shift+P).
1. Search for **Python: Create Environment**.
1. Select **Venv**.
1. Select a Python interpreter. Choose 3.10 or later.

It can take a minute to set up. If you run into problems, see [Python environments in VS Code](https://code.visualstudio.com/docs/python/environments).

### Install packages

In [6]:
! pip install -r requirements.txt --quiet

You should consider upgrading via the '/Users/jayakrishnamandivarapu/Downloads/Azure-rag/.venv/bin/python3 -m pip install --upgrade pip' command.


## Import required libraries and environment variables

In [2]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

load_dotenv(override=True) # take environment variables from .env.

# The following variables from your .env file are used in this notebook
endpoint = ''
credential = AzureKeyCredential('') #if len(os.environ["AZURE_SEARCH_ADMIN_KEY"]) > 0 else DefaultAzureCredential()
index_name = 'azure-rag-aisearch'


## Create your search index

Create your search index schema and vector search configuration. If you get an error, check the search service for available quota and check the .env file to make sure you're using a unique search index name.

In [4]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch,
    SearchIndex
)


# Create a search index
index_client = SearchIndexClient(
    endpoint=endpoint, credential=credential)
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="question", type=SearchFieldDataType.String),
    SearchableField(name="answer", type=SearchFieldDataType.String),
    SearchField(name="questionVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=384, vector_search_profile_name="myHnswProfile"),
    SearchField(name="answerVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=384, vector_search_profile_name="myHnswProfile"),
]

# Configure the vector search configuration  
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="myHnsw"
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
        )
    ]
)



semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="question"),
        content_fields=[SemanticField(field_name="answer")]
    )
)

# Create the semantic settings with the configuration
semantic_search = SemanticSearch(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, semantic_search=semantic_search)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

#index_client.delete_index(index)


 azure-rag-aisearch created


## Insert Documents into Mongodb


In [7]:
# List of JSON objects with explicit _id field

import pymongo
import json

data = [
    {
        "id": 6111,#key ID is required by Azure AI Search for dicument injection
        "question": "What is Microsoft Word?",
        "answer": "Microsoft Word is a word processing software developed by Microsoft. It allows users to create, edit, and format documents such as letters, reports, resumes, and more."
    },
    {
        "id": 7111,
        "question": "What is Microsoft Excel?",
        "answer": "Microsoft Excel is a spreadsheet software developed by Microsoft. It is used for tasks such as storing, organizing, and manipulating data, as well as performing calculations, creating charts, and generating reports."
    },
    {
        "id": 8111,
        "question": "What is Microsoft PowerPoint?",
        "answer": "Microsoft PowerPoint is a presentation software developed by Microsoft. It enables users to create slideshows with text, images, videos, and animations, making it suitable for presentations, lectures, and meetings."
    },
    {
        "id": 9111,
        "question": "What is Microsoft Outlook?",
        "answer": "Microsoft Outlook is an email client and personal information manager developed by Microsoft. It allows users to manage email, contacts, calendars, tasks, and notes, and integrates with other Microsoft Office applications."
    },
    {
        "id": 10111,
        "question": "What is Microsoft Teams?",
        "answer": "Microsoft Teams is a collaboration platform developed by Microsoft. It combines workplace chat, video meetings, file storage, and application integration, providing a hub for teamwork within organizations."
    },
    {
        "id": 11111,
        "question": "What is Microsoft Azure?",
        "answer": "Microsoft Azure is a cloud computing platform and services developed by Microsoft. It offers a wide range of cloud services, including computing, storage, analytics, networking, and more, enabling businesses to build, deploy, and manage applications and services through Microsoft's global network of data centers."
    }
]



# MongoDB connection string
connection_string = ""
client = pymongo.MongoClient(connection_string)

# Database name and collection name
db = client["azure-rag"]
collection = db["qa-pairs"]

# Upload each JSON object to MongoDB
for document in data:
  print(document)
  collection.insert_one(document)

print("Documents uploaded successfully.")

# List all databases
print("Databases:")
for db in client.list_databases():
    print(f"- {db['name']}")

# List all collections in each database
for db_name in client.list_database_names():
    print(f"\nCollections in database '{db_name}':")
    db = client[db_name]
    for collection_name in db.list_collection_names():
        print(f"- {collection_name}")


/var/folders/rj/9_rfks_s4q9bxkt8hfth_z580000gn/T/ipykernel_73810/1692447351.py:43: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  client = pymongo.MongoClient(connection_string)


{'id': 6111, 'question': 'What is Microsoft Word?', 'answer': 'Microsoft Word is a word processing software developed by Microsoft. It allows users to create, edit, and format documents such as letters, reports, resumes, and more.'}
{'id': 7111, 'question': 'What is Microsoft Excel?', 'answer': 'Microsoft Excel is a spreadsheet software developed by Microsoft. It is used for tasks such as storing, organizing, and manipulating data, as well as performing calculations, creating charts, and generating reports.'}
{'id': 8111, 'question': 'What is Microsoft PowerPoint?', 'answer': 'Microsoft PowerPoint is a presentation software developed by Microsoft. It enables users to create slideshows with text, images, videos, and animations, making it suitable for presentations, lectures, and meetings.'}
{'id': 9111, 'question': 'What is Microsoft Outlook?', 'answer': 'Microsoft Outlook is an email client and personal information manager developed by Microsoft. It allows users to manage email, contac

## Perform a vector similarity search

This example shows a pure vector search using the vectorizable text query, all you need to do is pass in text and your vectorizer will handle the query vectorization.

In [15]:
from azure.search.documents.models import VectorizedQuery
import requests
from azure.search.documents import SearchClient

search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential)

# Pure Vector Search
query = "Microsoft Word?"  
hf_token = "" #"get your token in http://hf.co/settings/tokens"
model_id = "sentence-transformers/all-MiniLM-L6-v2"

api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"
headers = {"Authorization": f"Bearer {hf_token}"}
response=requests.post(api_url, headers=headers, json={"inputs": query, "options":{"wait_for_model":True}})
embedding=response.json()  
vector_query = VectorizedQuery(vector=embedding, k_nearest_neighbors=3, fields="answerVector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["question", "answer"],
)  
  
for result in results:  
    print(f"Title: {result['question']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['answer']}")  


Title: What is Microsoft Word?
Score: 0.8596472
Content: Microsoft Word is a word processing software developed by Microsoft. It allows users to create, edit, and format documents such as letters, reports, resumes, and more.
Title: What is Microsoft PowerPoint?
Score: 0.66533184
Content: Microsoft PowerPoint is a presentation software developed by Microsoft. It enables users to create slideshows with text, images, videos, and animations, making it suitable for presentations, lectures, and meetings.
Title: What is Microsoft Excel?
Score: 0.662339
Content: Microsoft Excel is a spreadsheet software developed by Microsoft. It is used for tasks such as storing, organizing, and manipulating data, as well as performing calculations, creating charts, and generating reports.


## Perform an Exhaustive KNN exact nearest neighbor search

This example shows how you can exhaustively search your vector index regardless of what index you have, HNSW or ExhaustiveKNN. You can use this to calculate the ground-truth values.

In [ ]:
from azure.search.documents.models import VectorizedQuery
import requests
from azure.search.documents import SearchClient

search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential)

# Pure Vector Search
query = "Microsoft Word?"  
hf_token = "" #"get your token in http://hf.co/settings/tokens"
model_id = "sentence-transformers/all-MiniLM-L6-v2"

api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"
headers = {"Authorization": f"Bearer {hf_token}"}
response=requests.post(api_url, headers=headers, json={"inputs": query, "options":{"wait_for_model":True}})
embedding=response.json()  
vector_query = VectorizedQuery(vector=embedding, k_nearest_neighbors=3,exhaustive=True, fields="answerVector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["question", "answer"],
)  
  
for result in results:  
    print(f"Title: {result['question']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['answer']}")  


## Perform a Cross-Field Vector Search

This example shows a cross-field vector search that allows you to query multiple vector fields at the same time. Note, ensure that the same embedding model was used for the vector fields you decide to query.

In [16]:
# Pure Vector Search

# Pure Vector Search
query = "Microsoft Word?"  
hf_token = "" #"get your token in http://hf.co/settings/tokens"
model_id = "sentence-transformers/all-MiniLM-L6-v2"

api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"
headers = {"Authorization": f"Bearer {hf_token}"}
response=requests.post(api_url, headers=headers, json={"inputs": query, "options":{"wait_for_model":True}})
embedding=response.json()  
vector_query = VectorizedQuery(vector=embedding, k_nearest_neighbors=3,exhaustive=True, fields="questionVector, answerVector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["question", "answer"],
)  
  
for result in results:  
    print(f"Title: {result['question']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['answer']}")  



Title: What is Microsoft Word?
Score: 0.03333333507180214
Content: Microsoft Word is a word processing software developed by Microsoft. It allows users to create, edit, and format documents such as letters, reports, resumes, and more.
Title: What is Microsoft PowerPoint?
Score: 0.032786883413791656
Content: Microsoft PowerPoint is a presentation software developed by Microsoft. It enables users to create slideshows with text, images, videos, and animations, making it suitable for presentations, lectures, and meetings.
Title: What is Microsoft Excel?
Score: 0.032258063554763794
Content: Microsoft Excel is a spreadsheet software developed by Microsoft. It is used for tasks such as storing, organizing, and manipulating data, as well as performing calculations, creating charts, and generating reports.


## Perform a Pure Vector Search with a filter
This example shows how to apply filters on your index. Note, that you can choose whether you want to use Pre-Filtering (default) or Post-Filtering.

In [ ]:
from azure.search.documents.models import VectorFilterMode
from azure.search.documents.models import VectorizedQuery
import requests
from azure.search.documents import SearchClient

search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential)

# Pure Vector Search
query = "Microsoft Word?"  
hf_token = "" #"get your token in http://hf.co/settings/tokens"
model_id = "sentence-transformers/all-MiniLM-L6-v2"

api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"
headers = {"Authorization": f"Bearer {hf_token}"}
response=requests.post(api_url, headers=headers, json={"inputs": query, "options":{"wait_for_model":True}})
embedding=response.json()  
vector_query = VectorizedQuery(vector=embedding, k_nearest_neighbors=3,exhaustive=True, fields="answerVector")

results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    vector_filter_mode=VectorFilterMode.PRE_FILTER,
    filter="category eq 'Developer Tools'",
    select=["title", "content", "category"],
)
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  


## Perform a Semantic Hybrid Search

In [ ]:
from azure.search.documents.models import QueryType, QueryCaptionType, QueryAnswerType

# Semantic Hybrid Search
query = "what is azure sarch?"

embedding = client.embeddings.create(input=query, model=embedding_model_name).data[0].embedding
vector_query = VectorizedQuery(vector=embedding, k_nearest_neighbors=3, fields="contentVector", exhaustive=True)

results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["title", "content", "category"],
    query_type=QueryType.SEMANTIC, semantic_configuration_name='my-semantic-config', query_caption=QueryCaptionType.EXTRACTIVE, query_answer=QueryAnswerType.EXTRACTIVE,
    top=3
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(f"Title: {result['title']}")
    print(f"Reranker Score: {result['@search.reranker_score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")


RAG answering using Llama3

In [ ]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
from llama_index.prompts import PromptTemplate


# setting up the llm
llm = Ollama(model="llama3", request_timeout=120.0) 

# ====== Setup a query engine on the index previously created ======
Settings.llm = llm # specifying the llm to be used
query_engine = index.as_query_engine(streaming=True, similarity_top_k=4)

qa_prompt_tmpl_str = (
            "Context information is below.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "Given the context information above I want you to think step by step to answer the query in a crisp manner, incase case you don't know the answer say 'I don't know!'.\n"
            "Query: {query_str}\n"
            "Answer: "
            )

qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)
query_engine.update_prompts({"response_synthesizer:text_qa_template": qa_prompt_tmpl})

response = query_engine.query('What exactly is DSPy?')
print(response)